# Saving and Restoring Variables

## Saving

In [1]:
import tensorflow as tf


graph_save = tf.Graph()
with graph_save.as_default() as graph:
    # Create some variables.
    v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
    v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

    inc_v1 = v1.assign(v1+1)
    dec_v2 = v2.assign(v2-1)

    # Add an op to initialize the variables.
    init_op = tf.global_variables_initializer()

    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()
# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session(graph=graph_save) as sess:
  sess.run(init_op)
  # Do some work with the model.
  inc_v1.op.run()
  dec_v2.op.run()
  # Save the variables to disk.
  save_path = saver.save(sess, "/tmp/model.ckpt")
  print("v1: %s" % v1.eval())
  print("v2: %s" % v2.eval())
  print("Model saved in file: %s" % save_path)

v1: [ 1.  1.  1.]
v2: [-1. -1. -1. -1. -1.]
Model saved in file: /tmp/model.ckpt


Here are the file created by saver:

In [20]:
!ls  -l /tmp/model.ckpt*

-rw-rw-r-- 1 scott scott   32 Nov 20 14:52 /tmp/model.ckpt.data-00000-of-00001
-rw-rw-r-- 1 scott scott  143 Nov 20 14:52 /tmp/model.ckpt.index
-rw-rw-r-- 1 scott scott 4242 Nov 20 14:52 /tmp/model.ckpt.meta


## Restoring

we can also use `tf.train.Saver` to restore variable from checkpoint files

In [2]:
graph_restore = tf.Graph()
with graph_restore.as_default() as graph:
    # Create some variables.
    v1 = tf.get_variable("v1", shape=[3])
    v2 = tf.get_variable("v2", shape=[5])

    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()

with tf.Session(graph=graph_restore) as sess:
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored")
    #check the value
    print("v1: %s" % v1.eval())
    print("v2: %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
Model restored
v1: [ 1.  1.  1.]
v2: [-1. -1. -1. -1. -1.]


you can see that the value of v1 and v2 is the same as we saved.

## Chossing which variables to save and restore

if you do not pass any arguments to tf.train.Saver, the saver handles all variables in the graph. Each variable is saved under the name that was passed when the variable was created.

It is sometimes useful to explicitly specify names for variables in the checkpoint files. For example, you may have trained a model with a variable named "weights" whose value you want to restore into a variable named "params".

It is also sometimes useful to only save or restore a subset of the variables used by a model. For example, you may have trained a neural net with five layers, and you now want to train a new model with six layers that reuses the existing weights of the five trained layers. You can use the saver to restore the weights of just the first five layers

You can easily specify the names and variables to save or load by passing to the tf.train.Saver() constructor either of the following:

* A list of variables (which will be stored under their own names).
* A Python dictionary in which keys are the names to use and the values are the variables to manage.


In [8]:
graph_partial_save = tf.Graph()
with graph_partial_save.as_default() as graph:
    # Create some variables.
    v1 = tf.get_variable("v1", [3], initializer = tf.zeros_initializer)
    v2 = tf.get_variable("v2", [5], initializer = tf.zeros_initializer)

    # Add ops to save and restore only `v2` using the name "v2"
    saver = tf.train.Saver({"v2": v2})
    
with tf.Session(graph=graph_partial_save) as sess:
    # Initialize v1 since the saver will not.
    v1.initializer.run()
    saver.restore(sess, "/tmp/model.ckpt")
    print("v1 : %s" % v1.eval())
    print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt


INFO:tensorflow:Restoring parameters from /tmp/model.ckpt


v1 : [ 0.  0.  0.]
v2 : [-1. -1. -1. -1. -1.]


## inspect checkpoint

To inspect the variables in a checkpoint, you can use the `inspect_checkpint` library. Particulaly the `print_tensors_in_checkpoint_file` function. 

In [9]:
from tensorflow.python.tools import inspect_checkpoint

In [18]:
inspect_checkpoint.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name="", all_tensors=True)

tensor_name:  v1
[ 1.  1.  1.]
tensor_name:  v2
[-1. -1. -1. -1. -1.]


<tf.Tensor 'x:0' shape=(?, 784) dtype=float32>